In [1]:
import pdbufr
import sys
import traceback
 
from math import isnan

from eccodes import *
from ecmwf.opendata import Client
from math import isnan
from datetime import datetime, timedelta

import os
import birdy
import geopandas as gpd
import pandas as pd
import numpy as np
import xarray as xr
import requests

from ipywidgets import interact
import ipyleaflet
import ipywidgets as widgets

from Magics import macro as magics
from IPython.display import display
from Magics.macro import *

import warnings
warnings.filterwarnings("ignore")

from utils import *

In [2]:
## CODE TRIALS WITH FORECAST OF THE 20th AUGUST ##

start_date_forecast = datetime(datetime.now().year, datetime.now().month, datetime.now().day, hour=0)

In [3]:
## DOWNLOAD DATA FORECAST ##
# If too early in the morning the forecast of the current day cannot be donwloaded, need to fix this -> If raiseError: download forecast of the previous day
client = Client(source="azure")

start_date = start_date_forecast.strftime("%Y%m%d")

client.retrieve(
    date=int(start_date),
    time=0,
    stream="enfo",
    type="tf",
    step=240,
    target=f"data/tc_test_track_data.bufr",
);

20230823000000-240h-enfo-tf.bufr:   0%|          | 0.00/913k [00:00<?, ?B/s]

In [4]:
## DOWNLOAD OBSERVED DATA FROM IBTrACS ## 
url = 'https://www.ncei.noaa.gov/data/international-best-track-archive-for-climate-stewardship-ibtracs/v04r00/access/csv/ibtracs.ACTIVE.list.v04r00.csv'
r = requests.get(url, allow_redirects=True)
save_file = 'data/ibtracs.ACTIVE.list.v04r00.csv'
with open(save_file, 'wb') as f:
    f.write(r.content)

In [5]:
## LOAD TRACKS FORECAST DATA IN A DATAFRAME ##
# create_storms_df loads a file called tc_test_track_data.bufr
df_storms_forecast = create_storms_df()

## LOAD OBSERVED TRACKS IN A DATAFRAME
df_storms_observed = pd.read_csv('data/ibtracs.ACTIVE.list.v04r00.csv', header=[0,1])

print('Observed tracks: ', df_storms_observed.NAME.squeeze().unique().tolist())
print('Forecasted tracks: ', df_storms_forecast.stormIdentifier.unique())

Observed tracks:  ['DORA', 'LAN', 'FERNANDA', 'GREG', 'HILARY', 'GERT', 'EMILY', 'FRANKLIN', 'NOT_NAMED']
Forecasted tracks:  ['06L' '07L' '08L' '08W' '09L']


In [6]:
## FIND THE CORRESPONDENT OBSERVED STORMS TO THOSE PRESENT IN THE FORECAST ##
storms_pair = storms_pairing(df_storms_forecast, df_storms_observed)
print(storms_pair)

['07L-EMILY', '08L-FRANKLIN', '08W-DORA', '09L-NOT_NAMED']


In [7]:
## WIDGETS FOR INTERACTIVE MAPS PLOT ##

# Cyclone dropdown selection
cyclone = widgets.Dropdown(
    options = storms_pair,
    description = 'Active Storms:',
    disabled=False,
)
cyclone.style.description_width = '90px'

# Longitude slider
longitude = np.arange(-180,185,5)
longitude_slider = widgets.SelectionRangeSlider(
    options=longitude,
    index=(0, len(longitude)-1),
    description='Longitude:',
    orientation='horizontal',
    layout={'width': '400px'},
    disabled=False,
)

longitude_slider.style.description_width = '68px'
longitude_slider.style.handle_color = 'lightgreen'

# Latitude slider
latitude = np.arange(-90,95,5)
latitude_slider = widgets.SelectionRangeSlider(
    options=latitude,
    index=(0, len(latitude)-1),
    description='Latitude:',
    orientation='horizontal',
    layout={'width': '400px'},
    disabled=False,
)

latitude_slider.style.description_width = '58px'
latitude_slider.style.handle_color = 'lightgreen'

In [8]:
# widgets.interactive(plot_cyclone_tracks_magics, cyclone=cyclone, lat_boundaries=latitude_slider, lon_boundaries=longitude_slider)

In [9]:
# plot_cyclone_tracks_ipyleaflet('08L-FRANKLIN')

In [45]:
df_storm = df_storms_forecast[df_storms_forecast.stormIdentifier == '09L']
df_track = df_storm[df_storm.ensembleMemberNumber == 3]
initial_lat_lon = (df_storm.latitude.iloc[0], df_storm.longitude.iloc[0])
locations_f = forecast_tracks_locations(df_storm)

In [46]:
tc_track_map = ipyleaflet.Map(
        center=initial_lat_lon,
        basemap=ipyleaflet.basemaps.OpenStreetMap.France,
        zoom = 3.0,
        # scroll_wheel_zoom=True,
    )

colours = ["red", "blue", "green", "yellow", "purple", "orange", "cyan", "brown"]
tracks_layer_group = ipyleaflet.LayerGroup()
markers_layer_group = ipyleaflet.LayerGroup()
colour = 0

for locs in locations_f:
    if len(locs) > 0:
        track = ipyleaflet.Polyline(
            locations=locs,
            color= colours[colour],
            fill=False,
            weight=2,
            # name='Track %.02d' % i,
        )

        markers = [ipyleaflet.CircleMarker(location=loc, radius=1, color=colours[colour]) for loc in locs]

        colour += 1
        if colour == len(colours):
            colour = 0

        marker_layer = ipyleaflet.LayerGroup(layers=markers)

        tracks_layer_group.add_layer(track)
        markers_layer_group.add_layer(marker_layer)

layer_group_f = ipyleaflet.LayerGroup(layers=[tracks_layer_group, markers_layer_group], name='Forecasted Ensemble Tracks')

tc_track_map.add_layer(layer_group_f)

display(tc_track_map)

Map(center=[27.5, -99.8], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_ou…

In [12]:
# widgets.interactive(plot_cyclone_tracks_ipyleaflet, cyclone=cyclone)

In [51]:
print('paolo mangia \n merda')

paolo mangia 
 merda
